# GDAL 101

<a target="_blank" href="https://colab.research.google.com/github/mapninja/GDAL-101/blob/main/GDAL-101.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Overview of steps:
1. Verification: Verify GDAL installation.
2. Download the data: Placeholder for data download.
3. Check out the data in QGIS: Placeholder for QGIS.
4. OGR for Vector Data: Use GDAL to inspect vector data.
5. GDAL for Raster Data: Use GDAL to inspect raster data.
6. Converting Data: Convert data formats using GDAL.
7. Reprojecting Data: Reproject data using GDAL.
8. Querying Data: Query data using SQL with GDAL.
9. Clipping to create new data: Clip data using GDAL.
10. Previewing Data: Use Folium to preview data.

## Installation

Install necessary packages

In [ ]:
!pip install gdal folium

## Imports

In [4]:
from osgeo import gdal, ogr                     # Import the gdal and ogr modules from the osgeo package
import zipfile
import os
import folium
import geopandas as gpd

## Verification:

Verify GDAL installation

In [ ]:
gdal.UseExceptions()                            # Enable GDAL to use Python exceptions for error handling
print(gdal.__version__)                         # Print the version of the GDAL library

## Upload and Unzip the data in Colab

This assumes you are working in [Colab](https://colab.research.google.com/). First, you will need to place the `GDAL_101_data.zip` file in the `./contents` folder on Colab. This is the folder that is visible by default, in your Files Panel, in Colab.

In [ ]:
# Define the path to the zip file and the extraction directory (in Colab, right-click and select Copy Path to get this location)
zip_file_path = './GDAL_101_data.zip'
extract_to_path = './data'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Data extracted to {os.path.abspath(extract_to_path)}")

## Check out the data in Folium

In [ ]:


# Load the shapefile using geopandas
gdf = gpd.read_file(f"{extract_to_path}/zipcodes/zipcodes.shp")

# Create a map centered around the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=9)

# Add the shapefile layer to the map
folium.GeoJson(gdf).add_to(m)

# Display the map
m

## OGR for Vector Data

Inspect vector data

In [ ]:
!ogrinfo ./data/pools/pools.shp                        # Display information about the shapefile 'pools.shp'


In [ ]:
!ogrinfo ./data/zipcodes/zipcodes.shp                  # Display information about the shapefile 'zipcodes.shp'

In [ ]:
!ogrinfo -so ./data/zipcodes/zipcodes.shp zipcodes     # Display summary information about the 'zipcodes' layer in the 'zipcodes.shp' shapefile

## GDAL for Raster Data

Inspect raster data

In [ ]:
!gdalinfo ./data/dem10m/dem10m.dem


In [ ]:
!gdalinfo ./data/doqq.tif

## Converting Data

### Exporting original data into new formats

In [ ]:
!ogr2ogr --formats
!gdal_translate --formats

### shapefile → geojson

Convert shapefile to geojson

In [ ]:
!ogr2ogr -f geojson ./out/zipcodes.geojson ./data/zipcodes/zipcodes.shp
!ogrinfo -so ./out/zipcodes.geojson zipcodes

### GeoTIFF → georeferenced PNG

Convert GeoTIFF to georeferenced PNG

In [ ]:
!gdal_translate -of png ./data/doqq.tif ./out/converted_doqq.png
!gdalinfo ./out/converted_doqq.png

## Reprojecting Data

Reproject data using EPSG codes

In [ ]:
!ogr2ogr -t_srs EPSG:4326 ./out/reprojected_pools.shp ./data/pools/pools.shp
!gdalwarp -t_srs EPSG:4326 ./data/doqq.tif ./out/reprojected_doqq.tif

## Querying Data

Query data using SQL with GDAL

In [43]:
# Display the total count of features in the 'pools' shapefile
!ogrinfo ./data/pools/pools.shp -sql "SELECT COUNT(*) FROM pools"



INFO: Open of `./data/pools/pools.shp'
      using driver `ESRI Shapefile' successful.

Layer name: pools
Geometry: None
Feature Count: 1
Layer SRS WKT:
(unknown)
COUNT_*: Integer (0.0)
OGRFeature(pools):0
  COUNT_* (Integer) = 51073



In [44]:
# Display detailed information about the feature with FID (Feature ID) 1 in the 'pools' shapefile
!ogrinfo -q ./data/pools/pools.shp -sql "SELECT * FROM pools WHERE fid = 1"




Layer name: pools
OGRFeature(pools):1
  POOLS_2013 (Integer) = 2
  CREATED_BY (String) = AeroMetric
  CREATED_DA (Date) = 2013/10/01
  MODIFIED_B (String) = (null)
  MODIFIED_D (Date) = (null)
  SOURCE (String) = LiDAR 2012
  FEATURE (String) = Above ground
  ORIGIN_FEA (String) = Pools_2013
  Shape_Leng (Real) = 44.08991859810
  Shape_Area (Real) = 154.55091064300
  POLYGON Z ((3164378.2166 9997997.058 0,3164378.1782 9997996.3241 0,3164378.063 9997995.5984 0,3164377.8731 9997994.8884 0,3164377.6096 9997994.2024 0,3164377.2759 9997993.5479 0,3164376.8757 9997992.9314 0,3164376.4134 9997992.3605 0,3164375.8937 9997991.8408 0,3164375.3225 9997991.3783 0,3164374.7064 9997990.978 0,3164374.0515 9997990.6447 0,3164373.3655 9997990.3812 0,3164372.6559 9997990.1909 0,3164371.9302 9997990.0761 0,3164371.1962 9997990.0374 0,3164370.4623 9997990.0761 0,3164369.7366 9997990.1909 0,3164369.0269 9997990.3812 0,3164368.3406 9997990.6443 0,3164367.6861 9997990.978 0,3164367.0696 9997991.3783 0,31643

In [45]:
# Display the count of features in the 'pools' shapefile where the 'FEATURE' attribute is 'Above ground'
!ogrinfo -q ./data/pools/pools.shp -sql "SELECT COUNT(*) FROM pools WHERE FEATURE = 'Above ground'"


Layer name: pools
OGRFeature(pools):0
  COUNT_* (Integer) = 21550



### Saving query results to a new file

Create new files based on SQL queries

In [ ]:
!ogr2ogr ./out/austin_zips.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE name = 'Austin'"

In [ ]:
!ogr2ogr ./out/under_10mil.shp ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE SHAPE_AREA <10000000"

In [ ]:
!ogr2ogr -f geojson ./out/78756.geojson ./data/zipcodes/zipcodes.shp -sql "SELECT * FROM zipcodes WHERE ZIPCODE = '78756'"

## Clipping to create new data

Clip data using GDAL

In [ ]:
!ogr2ogr -clipsrc 78756.geojson 78756_pools.shp reprojected_pools.shp
!gdalwarp -cutline 78756.geojson doqq.tif 78756_doqq.tif
!gdal_translate -srcwin 0 0 1000 1000 -of USGSDEM dem10m/dem10m.dem clipped_dem.dem

## Previewing Data

Use Folium to preview data

In [ ]:
import folium
import geopandas as gpd

# Load the geojson file
gdf = gpd.read_file('zipcodes.geojson')

# Create a map centered around the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=12)

# Add the geojson layer to the map
folium.GeoJson('zipcodes.geojson').add_to(m)

# Display the map
m